In [1]:
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

/work/u7412932/myenv/.package_yolov7_v2_nchc_conda/envs/yolov7_v2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [ ]:
# pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116

In [31]:
source, weights, view_img, save_txt, imgsz, trace = 'pig_heatstress_v5-3/train/images/0525_4k_mp4-102_jpg.rf.2d80ad35717f9d602592f90a7ec3e1b8.jpg', '/home/u7412932/multiple_car/yolov7/runs/train/yolov7_multi/weights/best.pt', False,True, 640, False
save_img =  not source.endswith('.txt')  # save inference images


# Directories
save_dir = Path(increment_path(Path('runs/detect') / 'exp3', exist_ok=True))  # increment run
(save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

# Initialize
set_logging()
cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check img_size

if trace:
    model = TracedModel(model, device, 640)

if half:
    model.half()  # to FP16

# Second-stage classifier
classify = False
if classify:
    modelc = load_classifier(name='resnet101', n=2)  # initialize
    modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

# Set Dataloader
vid_path, vid_writer = None, None
# if webcam:
#     view_img = check_imshow()
#     cudnn.benchmark = True  # set True to speed up constant image size inference
#     dataset = LoadStreams(source, img_size=imgsz, stride=stride)
# else:
def check(source):
    dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    old_img_w = old_img_h = imgsz
    old_img_b = 1

    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Warmup
        if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
            old_img_b = img.shape[0]
            old_img_h = img.shape[2]
            old_img_w = img.shape[3]
            for i in range(3):
                model(img, augment=False)[0]

        # Inference
        t1 = time_synchronized()
        with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
            pred = model(img, augment=False)[0]
        t2 = time_synchronized()

        # Apply NMS
        pred = non_max_suppression(pred, 0.6, 0.4, classes=None, agnostic=False)
        t3 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        webcam=False
        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if True else (cls, *xywh)  # label format
                        print(txt_path)
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or view_img:  # Add bbox to image
                        label = f'{names[int(cls)]} {conf:.2f}'
                        # plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)

            # Print time (inference + NMS)
            print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                    print(f" The image with the result is saved in: {save_path}")
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)



Fusing layers... 


Model Summary: 314 layers, 36492560 parameters, 6194944 gradients, 103.2 GFLOPS


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


In [32]:
save_dir 

PosixPath('runs/detect/exp3')

In [27]:
# # 打开视频文件
# video = cv2.VideoCapture('/home/u7412932/yolo2024/20240221_2.mp4')

# # 检查视频是否成功打开
# if not video.isOpened():
#     print("无法打开视频文件")
#     exit()

# # 获取视频帧率
# fps = video.get(cv2.CAP_PROP_FPS)

# # 计算每秒保存一张图像所需的帧数间隔
# frame_interval = int(round(fps))

# # 初始化计数器和帧序号
# frame_count = 0
# save_count = 0

# # 逐帧读取视频并保存每秒一张图像
# while True:
#     # 读取帧
#     ret, im0 = video.read()
    
#     # 检查是否成功读取帧
#     if not ret:
#         break

#     # 检查是否达到保存帧的时间间隔
#     if frame_count % frame_interval == 0:
#         # print(frame_count)
#         cv2.imwrite(f'/home/u7412932/yolo2024/yoloo2024/image/frame_{frame_count}.jpg', im0)
#         # 保存帧为图像文件
#         # check(im0)
#     frame_count+=1

# video.release()

In [28]:
import os
filelist=os.listdir('/home/u7412932/yolo2024/yoloo2024/image')

In [29]:
len(filelist)

240

In [ ]:
for i in filelist:
    if i.endswith(".jpg"):
        # print('/home/u7412932/yolo2024/yoloo2024/image/'+i)
        check('/home/u7412932/yolo2024/yoloo2024/image/'+i)
        # print(i)
        # break
    # check(source)

In [34]:
import shutil

In [35]:
pwd

'/home/u7412932/multiple_car/yolov7'

In [36]:
for i in os.listdir('runs/detect/exp3'):
    if i.endswith(".jpg"):
        shutil.move('runs/detect/exp3/'+i, 'runs/detect/exp3/images/'+i)
        # check('runs/detect/exp'+i)

In [32]:
pwd

'/home/u7412932/multiple_car/yolov7'

In [41]:
mv /home/u7412932/multiple_car/yolov7/runs/detect/exp4 /home/u7412932/yolo2024/yoloo2024/afterlabel

mv: preserving permissions for '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp4/.ipynb_checkpoints/frame_1475-checkpoint.jpg': Operation not permitted
mv: preserving permissions for '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp4/.ipynb_checkpoints/frame_2250-checkpoint.jpg': Operation not permitted
mv: preserving permissions for '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp4/.ipynb_checkpoints/frame_1550-checkpoint.jpg': Operation not permitted
mv: preserving permissions for '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp4/.ipynb_checkpoints/frame_0-checkpoint.jpg': Operation not permitted
mv: preserving permissions for '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp4/.ipynb_checkpoints/frame_1275-checkpoint.jpg': Operation not permitted
mv: preserving permissions for '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp4/.ipynb_checkpoints/frame_1200-checkpoint.jpg': Operation not permitted
mv: preserving permissions for '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp4/.ip

In [40]:
# rm -r /home/u7412932/yolo2024/yoloo2024/afterlabel/exp3

rm: cannot remove '/home/u7412932/yolo2024/yoloo2024/afterlabel/exp3': Directory not empty
